In [81]:
import pandas as pd
import numpy as np
from scipy import stats
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KDTree
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

import statsmodels.api as sm
import statsmodels.formula.api as smf 

rng = np.random.RandomState(0)

In [82]:
# Initialisierung von DataFrames und Data Cleaning
df = pd.read_csv('Laptop-Preise.csv', sep=';', decimal=',')
df = df[df.extern_Schnittstellen != 2300] # Ausreißer löschen
df = df.reset_index(drop=True) # Reset Index
df.drop(['Betriebssystem_OHNE', 'Betriebssystem_Mac', 'Marke_Dell'], axis=1) # Mac und Marke_Apple sind identische Merkmale

df_noPrice = df.drop('Preis', axis=1)
imputed_simul_knn = pd.DataFrame(columns=['Mean KNN_1', 'MSE KNN_1', 'SE KNN_1', 'Alpha KNN_1', 'Mean KNN_3', 'MSE KNN_3', 'SE KNN_3', 'Alpha KNN_3', 'Mean KNN_5', 'MSE KNN_5', 'SE KNN_5', 'Alpha KNN_5'])
imputed_stats_knn = pd.DataFrame(columns=['Mean KNN_1', 'MSE KNN_1', 'SE KNN_1', 'Alpha KNN_1', 'Mean KNN_3', 'MSE KNN_3', 'SE KNN_3', 'Alpha KNN_3', 'Mean KNN_5', 'MSE KNN_5', 'SE KNN_5', 'Alpha KNN_5'], index = np.arange(0.1, 1, 0.1))

imputed_simul_ols = pd.DataFrame(columns=['Mean', 'MSE OLS', 'SE OLS', 'Alpha OLS'])
imputed_stats_ols = pd.DataFrame(columns=['Mean', 'MSE OLS', 'SE OLS', 'Alpha OLS'], index = np.arange(0.1, 1, 0.1))

no_imputation = pd.DataFrame(columns = ['SE', 'Alpha'])


# Skalierung (Standardisierung) von df_noPrice
col_names = df_noPrice.columns
scaler = StandardScaler().fit(df_noPrice.values)
df_noPrice = scaler.transform(df_noPrice.values)
df_noPrice = pd.DataFrame(df_noPrice, columns=col_names)


# Skalierung (Standardisierung) von df mit Preis (Preis ist unverändert)
df_std = df_noPrice.copy()
df_std.insert(0, 'Preis', df['Preis'])

# Wahrer Mittelwert:
true_mean = df['Preis'].mean()

In [83]:
def contains_mean(values):
    return 1 if (true_mean > (np.mean(values) - 1.95996 * stats.sem(values))) and (true_mean < (np.mean(values) + 1.95996 * stats.sem(values))) else 0

In [84]:
# OLS Model 
# fitting the model 
model = smf.ols(formula='Preis ~ '+' + '.join(df_noPrice.columns), data = df_std).fit() 
print(model.summary()) 

# OLS Model 
# fitting the model 
model = sm.OLS(df['Preis'], sm.add_constant(df_noPrice)).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Preis   R-squared:                       0.871
Model:                            OLS   Adj. R-squared:                  0.867
Method:                 Least Squares   F-statistic:                     234.5
Date:                Sun, 14 Jan 2024   Prob (F-statistic):               0.00
Time:                        22:27:29   Log-Likelihood:                -7279.4
No. Observations:                1038   AIC:                         1.462e+04
Df Residuals:                    1008   BIC:                         1.477e+04
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept   

In [85]:
train_values, test_values, train_labels, test_labels = train_test_split(df_noPrice, df['Preis'], test_size=0.01)

def del_ran(df_exog, labels, chance):
    rand_array = np.random.rand(df_exog.shape[0])
    delete_entries = rand_array < chance
    keep_entries = rand_array >= chance
    
    return [df_exog[delete_entries], labels[delete_entries], df_exog[keep_entries], labels[keep_entries]]

In [86]:
def impute_ols(test_values, test_labels, train_values, train_labels):
    
    # Um zu vermeiden, dass die Matrix am Ende nicht mehr den vollen Rang besitzt, fügen wir einen minimalen Jitter hinzu.
    # So kann die Regression wie gehabt durchgeführt werden. Die Jitter sollte die Werte nicht nennenswert beeinflussen,
    # Aber die Matrix wird auf alle Fälle ihren vollen Range behalten.
    random_values = np.random.uniform(low=-0.001, high=0.001, size=train_values.shape)

    # Add the random values to the original DataFrame
    train_values = train_values + random_values

    # OLS Model
    # fitting the model 
    model = sm.OLS(train_labels, sm.add_constant(train_values)).fit() 

    imputed_values = model.predict(exog = sm.add_constant(test_values, has_constant='add')).tolist()
    
    return [np.mean(list(train_labels) + imputed_values), np.mean((imputed_values-test_labels)**2), stats.sem(list(train_labels) + imputed_values), contains_mean(list(train_labels) + imputed_values)]

In [87]:
def impute_knn(test_values, test_labels, train_values, train_labels):
    
    tree = KDTree(train_values.values, leaf_size=5)

    imputed_values_knn_1 = []
    imputed_values_knn_3 = []
    imputed_values_knn_5 = []

    for index, entry in enumerate(test_values.values):
 
        dist, ind = tree.query([entry], k=5)
        ind = ind[0]

        current_impute_knn_1 = np.mean(train_labels.values[ind][0])
        current_impute_knn_3 = np.mean(train_labels.values[ind][:3])
        current_impute_knn_5 = np.mean(train_labels.values[ind])

        imputed_values_knn_1.append(current_impute_knn_1)
        imputed_values_knn_3.append(current_impute_knn_3)
        imputed_values_knn_5.append(current_impute_knn_5)

    mean_knn_1 = np.mean(list(train_labels.values)+imputed_values_knn_1)
    mean_knn_3 = np.mean(list(train_labels.values)+imputed_values_knn_3)
    mean_knn_5 = np.mean(list(train_labels.values)+imputed_values_knn_5)

    mse_knn_1 = np.mean((test_labels.values - imputed_values_knn_1)**2)
    mse_knn_3 = np.mean((test_labels.values - imputed_values_knn_3)**2)
    mse_knn_5 = np.mean((test_labels.values - imputed_values_knn_5)**2)

    sem_knn_1 = stats.sem(list(train_labels.values)+imputed_values_knn_1)
    sem_knn_3 = stats.sem(list(train_labels.values)+imputed_values_knn_3)
    sem_knn_5 = stats.sem(list(train_labels.values)+imputed_values_knn_5)

    alpha_knn_1 = contains_mean(list(train_labels.values)+imputed_values_knn_1)
    alpha_knn_3 = contains_mean(list(train_labels.values)+imputed_values_knn_3)
    alpha_knn_5 = contains_mean(list(train_labels.values)+imputed_values_knn_5)


    return [mean_knn_1, mse_knn_1, sem_knn_1, alpha_knn_1, mean_knn_3, mse_knn_3, sem_knn_3, alpha_knn_3, mean_knn_5, mse_knn_5, sem_knn_5, alpha_knn_5]

In [88]:
def simul_knn():
    for c in np.arange(0.1, 1, 0.1):

        imputed_simul_knn = pd.DataFrame(columns=['Mean KNN_1', 'MSE KNN_1', 'SE KNN_1', 'Alpha KNN_1', 'Mean KNN_3', 'MSE KNN_3', 'SE KNN_3', 'Alpha KNN_3', 'Mean KNN_5', 'MSE KNN_5', 'SE KNN_5', 'Alpha KNN_5'])

        for i in tqdm(range(1000)):
            temp = del_ran(df_exog = df_noPrice, labels = df['Preis'], chance = c)
            imputed_simul_knn.at[i] = impute_knn(temp[0], temp[1], temp[2], temp[3])

        return_values = []
        
        for column in imputed_simul_knn.columns:
            column_mean = np.mean(imputed_simul_knn[column].to_list())
            return_values.append(column_mean)
        # print(return_values)

        imputed_stats_knn.loc[c] =  return_values
    imputed_stats_knn

simul_knn()
imputed_stats_knn

100%|██████████| 1000/1000 [01:48<00:00,  9.18it/s]


,Mean KNN_1,MSE KNN_1,SE KNN_1,Alpha KNN_1,Mean KNN_3,MSE KNN_3,SE KNN_3,Alpha KNN_3,Mean KNN_5,MSE KNN_5,SE KNN_5,Alpha KNN_5
0.1,1332.39765,93408.961937,23.183238,1.0,1330.971138,76197.968787,23.046475,1.0,1330.038586,82065.999054,22.977208,1.0
0.2,1330.197019,96971.940264,23.119798,1.0,1327.395108,80420.641857,22.828853,1.0,1324.841444,86875.412974,22.687409,1.0
0.3,1327.623636,103856.786167,23.045989,1.0,1323.064113,86857.568804,22.571958,1.0,1318.782176,92821.119892,22.36113,1.0
0.4,1324.525763,109035.988482,22.946233,1.0,1317.620255,92777.38478,22.282596,1.0,1311.583211,97972.883297,22.007975,0.994
0.5,1320.196676,117051.701021,22.819841,1.0,1310.696391,99626.264788,21.945177,0.979,1304.068022,103486.304498,21.60695,0.901
0.6,1314.519275,128344.203187,22.688093,0.97,1301.694474,108812.451766,21.576704,0.801,1295.459643,110933.573714,21.174708,0.59
0.7,1306.475758,143866.636922,22.46324,0.848,1292.315696,119703.573814,21.124418,0.491,1287.250765,120506.326084,20.614482,0.334
0.8,1293.883596,167422.189149,22.206274,0.568,1280.540858,136644.855385,20.503769,0.281,1277.803152,138506.464086,19.753764,0.231
0.9,1281.477728,213492.838941,21.884303,0.393,1270.655029,176444.555519,19.233926,0.262,1261.458282,184960.447335,17.614519,0.183


In [89]:
def simul_ols():
    
    for c in np.arange(0.1, 1, 0.1):
        
        imputed_simul_ols = pd.DataFrame(columns=['Mean', 'MSE OLS', 'SE OLS', 'Alpha OLS'])

        for i in tqdm(range(1000)):
            temp = del_ran(df_exog = df_noPrice, labels = df['Preis'], chance = c)
            imputed_simul_ols.at[i] = impute_ols(temp[0], temp[1], temp[2], temp[3])

        return_values = []

        for column in imputed_simul_ols.columns:
            column_mean = np.mean(imputed_simul_ols[column].to_list())
            return_values.append(column_mean)
        # print(return_values)

        imputed_stats_ols.loc[c] =  return_values
    imputed_stats_ols

simul_ols()

100%|██████████| 1000/1000 [00:18<00:00, 54.27it/s]


Bei der Simulation wird irgendwann ein Fehler geworfen. Das liegt daran, dass, wenn zu viele Einträge gelöscht werden, die Matrix ihren vollen Rang verliert. Einige Dummy-Variablen sind relativ selten. Wenn nun alle Einträge mit der Dummy-Variable auf 1 (oder 0) gelöscht werden, dann wird wegen der linearen Abhängigkeit zur Konstante diese Variable gelöscht - so vermeiden wir Kollinarität.
Wenn dann aber nun im Test-Datensatz die Variable in ihrer Ausprägung 0 (oder 1) besitzt, dann können haben wir dafür keinen passenden Regressionskoeffizienten. Es werden dann 33 Variablen als Input geliefert, obwohl wir nur 32 Regressionskoeffizienten haben. Anschließend wird eine Fehlermeldung geworfen, dass der Eintrag, den wir vorhersagen wollen die falsche shape hat (32 statt 33). 
Es stellt sich zudem an diesem Punkt außerdem die Frage, wie sinnvoll es ist Variablen imputieren zu wollen, bei denen 70% der Einträge fehlen.

Bei den Auswertungen ist zu beachten, dass der wahre Standardfehler von Preis bei 23.2318 liegt.
Wir können also deutlich erkennen, dass jede Imputation der fehlenden Werte den Standardfehler künstlich verringert.

Es ist auch relativ deutlich klar, dass ein höherer k-Wert für KNN dazu führt, dass der Standardfehler weiter sinkt. Das liegt daran, dass bei einem höheren k-Wert der Mittelwert von einem größeren Teil des Datensatzes genommen wird. Dadurch wird der Mittelwert (oder zumindest Mittelwert-nahe Werte) imputiert.

Um zu vermeiden, dass die OLS-Regression einen Fehler wirft, wenn die Matrix nicht den vollen Rang besitzt, haben wir einen Jitter hinzugefügt. Bei jeder Auspräogung wird eine Zufallsvariable von -0.001 bis 0.001 hinzugefügt. Das sollte das Problem beheben, ohne zu viel Bias zu der Simulation hinzuzufügen.

Es treten jedoch Probleme auf, wenn wirklich große Teile des Datensatzes gelöscht werden. Dadurch fällt die Regression (wahrscheinlich) mehr auf den Jitter als erklärende Variable zurück und verzerrt alles unglaublich.

In [91]:
pd.concat([imputed_stats_ols, imputed_stats_knn], axis=1).to_csv('MCAR_Simulation_Normal')
pd.concat([imputed_stats_ols, imputed_stats_knn], axis=1)

,Mean,MSE OLS,SE OLS,Alpha OLS,Mean KNN_1,MSE KNN_1,SE KNN_1,Alpha KNN_1,Mean KNN_3,MSE KNN_3,SE KNN_3,Alpha KNN_3,Mean KNN_5,MSE KNN_5,SE KNN_5,Alpha KNN_5
0.1,1333.836671,77610.562371,23.07825,1.0,1332.39765,93408.961937,23.183238,1.0,1330.971138,76197.968787,23.046475,1.0,1330.038586,82065.999054,22.977208,1.0
0.2,1333.754517,77605.323178,22.934433,1.0,1330.197019,96971.940264,23.119798,1.0,1327.395108,80420.641857,22.828853,1.0,1324.841444,86875.412974,22.687409,1.0
0.3,1333.736907,77832.601758,22.774533,1.0,1327.623636,103856.786167,23.045989,1.0,1323.064113,86857.568804,22.571958,1.0,1318.782176,92821.119892,22.36113,1.0
0.4,1334.071176,78261.723418,22.649496,1.0,1324.525763,109035.988482,22.946233,1.0,1317.620255,92777.38478,22.282596,1.0,1311.583211,97972.883297,22.007975,0.994
0.5,1331.092004,1068400.558672,23.1893,0.998,1320.196676,117051.701021,22.819841,1.0,1310.696391,99626.264788,21.945177,0.979,1304.068022,103486.304498,21.60695,0.901
0.6,1334.39244,4586875.502512,25.835868,0.993,1314.519275,128344.203187,22.688093,0.97,1301.694474,108812.451766,21.576704,0.801,1295.459643,110933.573714,21.174708,0.59
0.7,1337.474972,25825946.48704,39.140143,0.977,1306.475758,143866.636922,22.46324,0.848,1292.315696,119703.573814,21.124418,0.491,1287.250765,120506.326084,20.614482,0.334
0.8,1308.478623,224613769.77313,133.800828,0.859,1293.883596,167422.189149,22.206274,0.568,1280.540858,136644.855385,20.503769,0.281,1277.803152,138506.464086,19.753764,0.231
0.9,1340.422629,1691304278.949417,615.831171,0.506,1281.477728,213492.838941,21.884303,0.393,1270.655029,176444.555519,19.233926,0.262,1261.458282,184960.447335,17.614519,0.183
